In [ ]:
!pip install transformers --upgrade



In [ ]:
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.2 MB/s eta 0:00:00


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the tokenizer and model
model_name = "t5-small"  # You can use "t5-small" or other variants
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [1]:
import pandas as pd
df = pd.read_excel('/content/Copy of Sample_Insights_from_MeetMinutes(1242).xlsx')




In [2]:
df.columns

Index(['og_transcript', 'file_name', 'language', 'participant', 'output'], dtype='object')

In [ ]:
import json
from transformers import T5Tokenizer

# Initialize the T5 tokenizer
model_name = "t5-small"  # You can use "t5-small" or other variants
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Define a function to format data
def format_data(text, summary, tokenizer, max_input_length=512, max_target_length=150):
    # Tokenize the input and target texts
    inputs = tokenizer.encode("summarize: " + text, max_length=max_input_length, truncation=True, padding='max_length')
    targets = tokenizer.encode(summary, max_length=max_target_length, truncation=True, padding='max_length')

    return {
        "input_ids": inputs,
        "attention_mask": [1] * len(inputs),  # 1s for input tokens
        "decoder_input_ids": targets[:-1],    # Target text without the [EOS] token
        "labels": targets[1:]                 # Target text without the [CLS] token
    }

# Format the entire dataset
formatted_data = []

for index, row in df.iterrows():
    input_text = row['og_transcript']
    output_json = json.loads(row['output'])
    target_summary = output_json['summary']['S']  # Assuming 'summary' is the key in your target JSON
    formatted_instance = format_data(input_text, target_summary, tokenizer)
    formatted_data.append(formatted_instance)

# Now, you have a list of dictionaries, where each dictionary contains input and target token IDs
# suitable for fine-tuning your T5 model.


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW

# Initialize the T5 tokenizer and model
model_name = "t5-small"  # You can use "t5-small" or other variants
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Convert the formatted data into PyTorch tensors
input_ids = torch.tensor([example["input_ids"] for example in formatted_data])
attention_mask = torch.tensor([example["attention_mask"] for example in formatted_data])
decoder_input_ids = torch.tensor([example["decoder_input_ids"] for example in formatted_data])
labels = torch.tensor([example["labels"] for example in formatted_data])

# Create a TensorDataset
dataset = TensorDataset(input_ids, attention_mask, decoder_input_ids, labels)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Define batch size and create data loaders
batch_size = 4  # Adjust this based on your available GPU memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 3  # Adjust the number of training epochs as needed

# Fine-tuning loop
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, decoder_input_ids, labels = batch
        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, decoder_input_ids, labels = batch
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids,
                labels=labels
            )
            total_val_loss += outputs.loss.item()

    average_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {average_val_loss:.4f}")

# Save the fine-tuned model
model.save_pretrained("fine-tuned-t5")
tokenizer.save_pretrained("fine-tuned-t5")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Validation Loss: 6.4920
Epoch 2/3, Validation Loss: 5.9135
Epoch 3/3, Validation Loss: 5.5340


('fine-tuned-t5/tokenizer_config.json',
 'fine-tuned-t5/special_tokens_map.json',
 'fine-tuned-t5/spiece.model',
 'fine-tuned-t5/added_tokens.json')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("fine-tuned-t5")
tokenizer = T5Tokenizer.from_pretrained("fine-tuned-t5")

# Function to generate a summary based on user input
def generate_summary(prompt, transcript, max_length=150, min_length=30, num_beams=4):
    input_text = prompt + " summarize: " + transcript
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)

    # Generate the summary
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=min_length, num_beams=num_beams, length_penalty=2.0, num_return_sequences=1)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary



In [ ]:
# Example usage:
user_prompt = "Please summarize the following discussion"
transcript_text = "This is the transcript of the discussion that needs to be summarized. " \
                  "During the discussion, participants talked about various topics, " \
                  "including the importance of AI in healthcare, challenges in " \
                  "implementing AI solutions, and potential future advancements. " \
                  "They also discussed the need for collaboration between " \
                  "healthcare professionals and AI experts."

generated_summary = generate_summary(user_prompt, transcript_text)
print("Generated Summary:")
print(generated_summary)


Generated Summary:
This is the transcript of the discussion that needs to be summarized. participants talked about various topics, including the importance of AI in healthcare, challenges in implementing AI solutions, and potential future advancements.


In [ ]:
df['og_transcript'][3]

'"WEBVTT\\n\\n00:00:00.170 --> 00:00:20.800\\nSpeaker A: This, we click on the meeting link. We paste the link, we click on proceed. Whatever request you send, you are getting a bot ID back. You should use that bot ID for doing other tasks. All right, so currently are you saving the bot ID in some way?\\n\\n00:00:22.450 --> 00:00:26.246\\nSpeaker B: No. So we are not saving it in our storage right now.\\n\\n00:00:26.348 --> 00:00:26.758\\nSpeaker A: Okay.\\n\\n00:00:26.844 --> 00:00:33.430\\nSpeaker B: But we are giving that response that bot has been sent after we so.\\n\\n00:00:33.500 --> 00:01:35.510\\nSpeaker A: You are sending a response like this, request like this and you will get a response bot ID and there will be an ID of the bot. Just save it and with the bot ID you will be able to do further things. The first part is discussing this only sending a bot to the meeting. You send the request and you will get the bot ID as a response back. Now, to check the status of the bot, wh

In [ ]:
# Assuming your transcript is in the format you provided earlier
transcript_text =df['og_transcript'][3]

# Split the transcript into chunks based on speaker turns
transcript_lines = transcript_text.strip().split('\n')

speaker_turns = []
current_speaker_turn = []

for line in transcript_lines:
    if line.startswith("Speaker"):
        if current_speaker_turn:
            speaker_turns.append("\n".join(current_speaker_turn))
        current_speaker_turn = [line]
    else:
        current_speaker_turn.append(line)

# Add the last speaker turn
if current_speaker_turn:
    speaker_turns.append("\n".join(current_speaker_turn))

# Now, `speaker_turns` contains chunks based on speaker turns
for i, chunk in enumerate(speaker_turns):
    print(f"Chunk {i + 1}:")
    print(chunk)
    print("\n")


Chunk 1:
"WEBVTT\n\n00:00:00.170 --> 00:00:20.800\nSpeaker A: This, we click on the meeting link. We paste the link, we click on proceed. Whatever request you send, you are getting a bot ID back. You should use that bot ID for doing other tasks. All right, so currently are you saving the bot ID in some way?\n\n00:00:22.450 --> 00:00:26.246\nSpeaker B: No. So we are not saving it in our storage right now.\n\n00:00:26.348 --> 00:00:26.758\nSpeaker A: Okay.\n\n00:00:26.844 --> 00:00:33.430\nSpeaker B: But we are giving that response that bot has been sent after we so.\n\n00:00:33.500 --> 00:01:35.510\nSpeaker A: You are sending a response like this, request like this and you will get a response bot ID and there will be an ID of the bot. Just save it and with the bot ID you will be able to do further things. The first part is discussing this only sending a bot to the meeting. You send the request and you will get the bot ID as a response back. Now, to check the status of the bot, what you 

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("fine-tuned-t5")
tokenizer = T5Tokenizer.from_pretrained("fine-tuned-t5")

# Define prompt templates for questions, notes, and action items
question_prompt = "Generate a question from the following transcript: "
note_prompt = "Generate a note from the following transcript: "
other_action_item_prompt = "Generate an action item for others from the following transcript: "
user_action_item_prompt = "Generate an action item for the selected participants from the following transcript: "

# Function to generate content based on prompts
def generate_content(prompt, transcript_chunk):
    input_text = transcript_chunk
    input_prompt = input_text + " " + prompt

    # Tokenize and generate content based on the prompt
    input_ids = tokenizer.encode(input_prompt, return_tensors="pt", max_length=512, truncation=True, padding=True)
    generated_ids = model.generate(input_ids, max_length=150, min_length=30, num_beams=4, length_penalty=2.0, num_return_sequences=1)
    generated_content = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return generated_content

# Initialize lists to store extracted information
questions = []
notes = []
other_action_items = []
user_action_items = []

# Process each transcript chunk
for chunk in transcript_chunks:
    # Extract questions
    question = generate_content(question_prompt, chunk)
    questions.append(question)

    # Extract notes
    note = generate_content(note_prompt, chunk)
    notes.append(note)

    # Extract other action items
    other_action_item = generate_content(other_action_item_prompt, chunk)
    other_action_items.append(other_action_item)

    # Extract user action items
    user_action_item = generate_content(user_action_item_prompt, chunk)
    user_action_items.append(user_action_item)

# Print the extracted information
print("Extracted Questions:")
for i, question in enumerate(questions):
    print(f"Question {i + 1}: {question}")

print("\nExtracted Notes:")
for i, note in enumerate(notes):
    print(f"Note {i + 1}: {note}")

print("\nExtracted Other Action Items:")
for i, other_action_item in enumerate(other_action_items):
    print(f"Other Action Item {i + 1}: {other_action_item}")

print("\nExtracted User Action Items:")
for i, user_action_item in enumerate(user_action_items):
    print(f"User Action Item {i + 1}: {user_action_item}")

Extracted Questions:
Question 1: During the discussion, participants talked about various t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from
Question 2: opics, including the importance of AI in healthcare, challenges in implementing AI solutions, and po Generate a question from the following transcript:
Question 3: also discussed the need for collaboration between healthcare profe Generate a question from the following transcript :
Question 4: Generieren Sie eine Frage aus dem folgenden Protokoll: ssionals, AI experts, ssionals, AI experts, ssionals, AI experts, ssionals, AI experts, ssionals,

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("fine-tuned-t5")
tokenizer = T5Tokenizer.from_pretrained("fine-tuned-t5")

# Define prompt templates for questions, notes, and action items
question_prompt = "Generate a question from the following transcript: "
note_prompt = "Generate a note from the following transcript: "
other_action_item_prompt = "Generate an action item for others from the following transcript: "
user_action_item_prompt = "Generate an action item for the selected participants from the following transcript: "

# Function to generate content based on prompts
def generate_content(prompt, transcript_chunk):
    input_text = transcript_chunk
    input_prompt = input_text + " " + prompt

    # Tokenize and generate content based on the prompt
    input_ids = tokenizer.encode(input_prompt, return_tensors="pt", max_length=512, truncation=True, padding=True)
    generated_ids = model.generate(input_ids, max_length=150, min_length=30, num_beams=4, length_penalty=2.0, num_return_sequences=1)
    generated_content = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return generated_content

# Initialize lists to store extracted information
questions = []
notes = []
other_action_items = []
user_action_items = []

# Process each transcript chunk
for chunk in transcript_chunks:
    # Extract questions
    question = generate_content(question_prompt, chunk)
    questions.append(question)

    # Extract notes
    note = generate_content(note_prompt, chunk)
    notes.append(note)

    # Extract other action items
    other_action_item = generate_content(other_action_item_prompt, chunk)
    other_action_items.append(other_action_item)

    # Extract user action items
    user_action_item = generate_content(user_action_item_prompt, chunk)
    user_action_items.append(user_action_item)

# Construct the final output dictionary
output_dict = {
    "questions": {"L": [{"S": q} for q in questions]},
    "summary": {"S": "Short summary of the meeting"},
    "notes": {"L": [{"S": n} for n in notes]},
    "otheractionitem": {"L": [{"S": oai} for oai in other_action_items]},
    "useractionitem": {"L": [{"S": uai} for uai in user_action_items]},
}
# Pretty-print the output_dict
formatted_output = json.dumps(output_dict, indent=4)
# Print the final output dictionary
print(output_dict)


{'questions': {'L': [{'S': 'During the discussion, participants talked about various t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from'}, {'S': 'opics, including the importance of AI in healthcare, challenges in implementing AI solutions, and po Generate a question from the following transcript:'}, {'S': 'also discussed the need for collaboration between healthcare profe Generate a question from the following transcript :'}, {'S': 'Generieren Sie eine Frage aus dem folgenden Protokoll: ssionals, AI experts, ssionals, AI experts, ssionals, AI experts, ssionals, AI experts, ssionals, AI experts

In [ ]:
output_dict

{'questions': {'L': [{'S': 'During the discussion, participants talked about various t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from the following transcript: t Generate a question from'},
   {'S': 'opics, including the importance of AI in healthcare, challenges in implementing AI solutions, and po Generate a question from the following transcript:'},
   {'S': 'also discussed the need for collaboration between healthcare profe Generate a question from the following transcript :'},
   {'S': 'Generieren Sie eine Frage aus dem folgenden Protokoll: ssionals, AI experts, ssionals, AI experts, ssionals, AI experts, ssionals, AI experts, ssionals, A